# 03_baseline_train
- Author: 
- Date: 2025-10-23
- Goal: Baseline CNN 학습 및 로깅
- Input: 
- Output: 
- Metrics: acc@val, loss@train
- Repro: seed=42, device=auto, config=../configs/


In [ ]:
# ============================================================
# 03_analysis_report.ipynb
#
# 목적:
#   - 02_train_classical_ml.ipynb 결과를 정리/분석하여
#     보고서와 발표 자료에 바로 사용할 수 있는 형태로 가공한다.
#
# 주요 기능:
#   1) metrics CSV 로드 및 Task별 모델 성능 비교 표 생성
#   2) 각 Task에서 Best Model 선택 (macro F1 기준)
#   3) Confusion Matrix / Feature Importance 결과 해석 가이드 작성
#   4) "결과 → 원인 → 의미 → 결론" 구조의 분석 문단 자동 생성
#
# 전제:
#   - 01, 02 노트북이 먼저 정상적으로 실행되어야 함
#   - results/metrics/, results/figures/ 이미 존재
# ============================================================

# ------------------------------------------------------------
# [Cell 1] 기본 설정 및 라이브러리 임포트
# ------------------------------------------------------------

import os  # 경로 제어
import numpy as np  # 수치 계산
import pandas as pd  # 테이블 데이터 로딩/처리
import matplotlib.pyplot as plt  # 시각화
import seaborn as sns  # heatmap 및 스타일

# 시각화 스타일: 보고서용으로 깔끔하게
plt.style.use("default")
plt.rcParams["font.family"] = "DejaVu Sans"  # 영어 기준 폰트
plt.rcParams["axes.unicode_minus"] = False

print("[OK] Imports ready.")

In [ ]:
# ------------------------------------------------------------
# [Cell 2] BASE_DIR 및 경로 설정
#   - 02와 동일한 방식: notebooks/에서 실행해도 동작하도록 처리
# ------------------------------------------------------------

cwd = os.getcwd()

if os.path.basename(cwd) == "notebooks":
    BASE_DIR = os.path.dirname(cwd)
else:
    BASE_DIR = cwd

METRICS_DIR = os.path.join(BASE_DIR, "results", "metrics")
FIGURES_DIR = os.path.join(BASE_DIR, "results", "figures")

summary_path = os.path.join(METRICS_DIR, "classical_ml_summary.csv")

print("[INFO] BASE_DIR   :", BASE_DIR)
print("[INFO] METRICS_DIR:", METRICS_DIR)
print("[INFO] FIGURES_DIR:", FIGURES_DIR)

In [ ]:
# ------------------------------------------------------------
# [Cell 3] metrics 파일 로드 검증
#   - 02 노트북이 제대로 실행되었는지 확인하는 단계
# ------------------------------------------------------------

if not os.path.exists(summary_path):
    raise FileNotFoundError(
        f"[ERROR] {summary_path} not found.\n"
        "Please run 02_train_classical_ml.ipynb first."
    )

summary_df = pd.read_csv(summary_path)

# 기대 컬럼 구조 체크
expected_cols = {
    "task",
    "model",
    "accuracy",
    "precision_macro",
    "recall_macro",
    "f1_macro",
}
missing = expected_cols - set(summary_df.columns)
if missing:
    raise ValueError(f"[ERROR] Missing columns in summary CSV: {missing}")

print("[OK] Loaded metrics summary.")
display(summary_df.head())

In [ ]:
# ------------------------------------------------------------
# [Cell 4] Task별 모델 성능 표 생성
#   - 각 Task에 대해 모델 성능을 보기 좋게 정리
#   - macro F1 기준 내림차순 정렬
# ------------------------------------------------------------

tasks = ["digit", "fg_color", "bg_color"]

task_tables = {}  # 나중에 보고서/PPT에 쓸 수 있도록 저장

for task in tasks:
    df_task = summary_df[summary_df["task"] == task].copy()
    if df_task.empty:
        print(f"[WARN] No rows for task={task} in summary.")
        continue

    # F1 기준 상위 모델부터 정렬
    df_task = df_task.sort_values(by="f1_macro", ascending=False)

    # 보기 좋은 컬럼 순서로 정리
    df_task = df_task[[
        "model",
        "accuracy",
        "precision_macro",
        "recall_macro",
        "f1_macro",
    ]]

    # 소수점 자리 보기 좋게 제한
    df_task = df_task.round(4)

    task_tables[task] = df_task

    print(f"\n===== Performance table for task: {task} =====")
    display(df_task)

In [ ]:
# ------------------------------------------------------------
# [Cell 5] Task별 Best Model 선정
#   - macro F1 기준으로 최고 성능 모델 선택
#   - 보고서에 바로 쓸 수 있는 dict 생성
# ------------------------------------------------------------

best_models = {}

for task in tasks:
    df_task = task_tables.get(task, None)
    if df_task is None or df_task.empty:
        continue

    # F1이 가장 높은 모델 1개 선택
    best_row = df_task.sort_values(by="f1_macro", ascending=False).iloc[0]
    best_models[task] = {
        "model": best_row["model"],
        "f1_macro": float(best_row["f1_macro"]),
        "accuracy": float(best_row["accuracy"]),
    }

print("\n[OK] Best models per task:")
for task, info in best_models.items():
    print(f"  - {task:9s}: {info['model']} "
          f"(F1={info['f1_macro']:.4f}, Acc={info['accuracy']:.4f})")


In [ ]:
# ------------------------------------------------------------
# [Cell 6] 성능 비교 시각화 (막대 그래프)
#   - 각 Task별로 모델별 F1-score 비교 바차트 생성
#   - 발표용/보고서용 Figure 자동 생성
# ------------------------------------------------------------

os.makedirs(FIGURES_DIR, exist_ok=True)

for task in tasks:
    df_task = task_tables.get(task, None)
    if df_task is None or df_task.empty:
        continue

    plt.figure(figsize=(6, 4))
    sns.barplot(
        data=df_task,
        x="model",
        y="f1_macro",
    )
    plt.title(f"Macro F1 by model - {task}")
    plt.ylim(0, 1.0)
    plt.ylabel("Macro F1")
    plt.xlabel("Model")
    plt.tight_layout()

    out_path = os.path.join(FIGURES_DIR, f"bar_f1_{task}.png")
    plt.savefig(out_path, dpi=200)
    plt.close()
    print(f"[OK] Saved F1 bar chart for {task} → {out_path}")


In [ ]:

# ------------------------------------------------------------
# [Cell 7] 분석 문단 자동 생성 함수
# ------------------------------------------------------------

def generate_task_analysis(task_name: str, table: pd.DataFrame, best_info: dict) -> str:
    """Task별 '결과→원인→의미→결론' 텍스트 생성."""

    if table is None or table.empty or best_info is None:
        return f"[{task_name}] No sufficient data to analyze.
"

    best_model = best_info["model"]
    best_f1 = best_info["f1_macro"]

    others = table[table["model"] != best_model]
    if not others.empty:
        others_mean_f1 = others["f1_macro"].mean()
        gap = best_f1 - others_mean_f1
    else:
        others_mean_f1 = None
        gap = None

    if task_name == "digit":
        task_desc = "10-class digit classification (0-9)"
        difficulty = "숫자 형태 정보가 뚜렷하여 비교적 안정적인 분류가 가능한 Task"
    elif task_name == "fg_color":
        task_desc = "7-class foreground color classification"
        difficulty = "전경 색상만 구분해야 하며, 배경/노이즈 영향으로 난도가 상승하는 Task"
    elif task_name == "bg_color":
        task_desc = "7-class background color classification"
        difficulty = "배경 색상만 반영되어야 하며, 전경과의 대비에 따라 혼동이 발생할 수 있는 Task"
    else:
        task_desc = task_name
        difficulty = ""

    lines = []
    lines.append(f"[Task 설명] {task_desc}")
    if difficulty:
        lines.append(f"[난이도 요약] {difficulty}")

    lines.append(
        f"[결과] 이 Task에서 가장 높은 성능을 기록한 모델은 **{best_model}** 이며,"
        f" macro F1 기준 **{best_f1:.4f}** 수준이다."
    )

    if others_mean_f1 is not None:
        lines.append(
            f"[비교] 나머지 모델들의 평균 F1은 약 **{others_mean_f1:.4f}** 수준으로,"
            f" 최고 성능 모델과의 격차는 약 **{gap:.4f}** 포인트이다."
        )

    if best_model in ["RandomForest", "XGBoost"]:
        lines.append(
            "[원인] 트리 기반 앙상블 모델은 픽셀 단위의 비선형 패턴과 색상 조합 정보를 동시에 학습할 수 있어, "
            "단일 트리(DecisionTree)에 비해 복잡한 결정 경계를 더 잘 표현한다."
        )
    elif best_model == "DecisionTree":
        lines.append(
            "[원인] 단일 트리는 해석력은 높지만 일반적으로 앙상블보다 성능이 낮은 편이다. "
            "본 Task에서 DecisionTree가 높은 성능을 보였다면, 데이터 패턴이 비교적 단순하거나 깊이 제약 설정이 잘 맞았다는 의미로 해석할 수 있다."
        )

    lines.append(
        "[의미] 본 결과는 모델 구조가 색상·형태 정보를 어떻게 활용하는지에 따라 성능 차이가 뚜렷하게 발생함을 보여준다. "
        "특히 앙상블/부스팅 계열 모델은 feature 간 상호작용까지 고려할 수 있기 때문에 Colored MNIST와 같은 복합 특성 데이터에서 우위를 가진다."
    )

    lines.append(
        "[결론] 최종적으로 이 Task에서는 "
        f"**{best_model}** 을 기준 모델로 채택하고, "
        "하이퍼파라미터 튜닝 및 추가적인 전처리 개선을 통해 성능을 보강하는 전략이 타당하다."
    )

    return "
".join(lines) + "
"



In [ ]:
# ------------------------------------------------------------
# [Cell 8] Task별 분석 문단 생성 및 표시
#   - 노트북 출력 내용을 그대로 보고서/노션에 복사해서 사용 가능
# ------------------------------------------------------------

analysis_blocks = {}

for task in tasks:
    table = task_tables.get(task, None)
    best_info = best_models.get(task, None)
    block = generate_task_analysis(task, table, best_info)
    analysis_blocks[task] = block

    print(f"\n==================== Analysis: {task} ====================")
    print(block)

In [ ]:

# ------------------------------------------------------------
# [Cell 9] 전체 프로젝트 요약 문단 생성
# ------------------------------------------------------------

def generate_overall_summary(best_models_dict):
    """3개 Task 결과를 종합한 high-level 결론."""

    model_counts = {}
    for info in best_models_dict.values():
        m = info["model"]
        model_counts[m] = model_counts.get(m, 0) + 1

    overall_best = max(model_counts.items(), key=lambda x: x[1])[0] if model_counts else None

    lines = []
    lines.append("[종합 결과 요약]")
    if overall_best:
        lines.append(
            f"- 세 Task 중 다수에서 최상위 F1을 기록한 모델은 **{overall_best}** 이며, Colored MNIST 환경에서 가장 일관된 성능을 보였다."
        )

    lines.append(
        "- Digit Task는 전통적인 숫자 인식 문제로 상대적으로 높은 정확도를 보였으며, Color 관련 Task(fg_color, bg_color)는 색상 조합과 대비에 따라 난이도가 상승하는 양상이 확인되었다."
    )

    lines.append(
        "- 트리 기반 앙상블(RandomForest, XGBoost)은 색상과 형태가 동시에 존재하는 입력에서 강인한 성능을 보였고, Feature Importance 시각화를 통해 모델이 실제로 숫자 영역 및 색상 채널에 집중하고 있음을 확인할 수 있다."
    )

    lines.append(
        "- DecisionTree는 해석 가능성과 빠른 추론 속도가 장점이지만, 앙상블 대비 복잡한 색상 패턴에서의 표현력 한계를 드러냈다. 이는 향후 모델 선택 시 단일 트리를 참고 지표로 활용하되, 최종 시스템은 앙상블/부스팅 계열로 구성해야 함을 시사한다."
    )

    lines.append(
        "- 전처리 과정에서의 폰트 합성, FG/BG 색상 라벨 정합성, 과도하지 않은 augmentation 전략이 전체 실험의 안정성을 높이는 핵심 요소로 작용하였다."
    )

    lines.append(
        "[최종 결론] 본 프로젝트에서는 트리 기반 앙상블과 부스팅(XGBoost)을 중심 모델로 채택하고, Colored MNIST와 유사한 구조의 데이터에서 전통적 ML 기법만으로도 충분히 경쟁력 있는 성능과 해석 가능성을 확보할 수 있음을 보였다."
    )

    return "
".join(lines)


overall_summary = generate_overall_summary(best_models)

print("
==================== Overall Summary ====================")
print(overall_summary)

